# Phased Array Antenna Modeling - Comprehensive Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/Phased-Array-Antenna-Model/blob/main/Phased_Array_Demo.ipynb)

This notebook demonstrates all features of the `phased_array` package:

1. **Performance**: Vectorized array factor computation (50-100x faster)
2. **Array Geometries**: Rectangular, circular, conformal, sparse arrays
3. **Beamforming**: Amplitude tapering, null steering, multi-beam
4. **Impairments**: Mutual coupling, phase quantization, element failures
5. **Visualization**: 2D/3D plots, UV-space representation

In [ ]:
# Install dependencies (uncomment for Colab)
# !pip install numpy matplotlib scipy plotly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

# Add package to path (for local development)
sys.path.insert(0, '.')

import phased_array as pa

print(f"phased_array version: {pa.__version__}")

## 1. Basic Array Factor Computation

Create a simple rectangular array and compute its radiation pattern.

In [ ]:
# Create a 16x16 rectangular array with half-wavelength spacing
Nx, Ny = 16, 16
dx, dy = 0.5, 0.5  # wavelengths
wavelength = 1.0   # normalized

geom = pa.create_rectangular_array(Nx, Ny, dx, dy, wavelength)
print(f"Array has {geom.n_elements} elements")

# Wavenumber
k = pa.wavelength_to_k(wavelength)

# Uniform weights (broadside beam)
weights = np.ones(geom.n_elements, dtype=complex)

# Compute pattern
theta, phi, pattern_dB = pa.compute_full_pattern(
    geom.x, geom.y, weights, k,
    n_theta=91, n_phi=361
)

# Plot contour
pa.plot_pattern_contour(
    np.rad2deg(theta), np.rad2deg(phi), pattern_dB,
    title="16x16 Array - Broadside Beam"
)
plt.show()

## 2. Beam Steering

Steer the beam to different angles using phase shifts.

In [ ]:
# Steer beam to theta=30 deg, phi=45 deg
theta0, phi0 = 30, 45  # degrees

weights_steered = pa.steering_vector(k, geom.x, geom.y, theta0, phi0)

# Compute pattern cuts
angles, E_plane, H_plane = pa.compute_pattern_cuts(
    geom.x, geom.y, weights_steered, k,
    theta0_deg=theta0, phi0_deg=phi0
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

pa.plot_pattern_2d(angles, E_plane, title=f"E-Plane Cut (phi={phi0}deg)", ax=ax1)
pa.plot_pattern_2d(angles, H_plane, title=f"H-Plane Cut (phi={phi0+90}deg)", ax=ax2)

plt.tight_layout()
plt.show()

print(f"E-plane HPBW: {pa.compute_half_power_beamwidth(angles, E_plane):.2f} deg")
print(f"H-plane HPBW: {pa.compute_half_power_beamwidth(angles, H_plane):.2f} deg")

## 3. Performance Comparison: Vectorized vs Loop

Demonstrate the speedup of vectorized computation.

In [ ]:
# Reference loop implementation (slow)
def array_factor_loop(theta, phi, x, y, weights, k):
    AF = np.zeros(theta.shape, dtype=complex)
    for i in range(len(x)):
        u = np.sin(theta) * np.cos(phi)
        v = np.sin(theta) * np.sin(phi)
        phase = k * (x[i] * u + y[i] * v)
        AF += weights[i] * np.exp(1j * phase)
    return AF

# Test grid
theta_test = np.linspace(0, np.pi/2, 91)
phi_test = np.linspace(0, 2*np.pi, 361)
theta_grid, phi_grid = np.meshgrid(theta_test, phi_test, indexing='ij')

# Time loop version
start = time.time()
AF_loop = array_factor_loop(theta_grid, phi_grid, geom.x, geom.y, weights_steered, k)
time_loop = time.time() - start

# Time vectorized version
start = time.time()
AF_vec = pa.array_factor_vectorized(theta_grid, phi_grid, geom.x, geom.y, weights_steered, k)
time_vec = time.time() - start

print(f"Loop time: {time_loop*1000:.1f} ms")
print(f"Vectorized time: {time_vec*1000:.1f} ms")
print(f"Speedup: {time_loop/time_vec:.1f}x")
print(f"Results match: {np.allclose(AF_loop, AF_vec)}")

## 4. Amplitude Tapering for Sidelobe Control

Apply various window functions to reduce sidelobes.

In [ ]:
# Compare different tapers
tapers = {
    'Uniform': np.ones(geom.n_elements),
    'Taylor -25dB': pa.taylor_taper_2d(Nx, Ny, sidelobe_dB=-25),
    'Taylor -35dB': pa.taylor_taper_2d(Nx, Ny, sidelobe_dB=-35),
    'Chebyshev -30dB': pa.chebyshev_taper_2d(Nx, Ny, sidelobe_dB=-30),
    'Hamming': pa.hamming_taper_2d(Nx, Ny),
}

patterns = {}
for name, taper in tapers.items():
    weights_tapered = weights_steered * taper
    angles, E_plane, _ = pa.compute_pattern_cuts(
        geom.x, geom.y, weights_tapered, k,
        theta0_deg=theta0, phi0_deg=phi0
    )
    patterns[name] = E_plane
    
    # Print efficiency
    eff = pa.compute_taper_efficiency(taper)
    loss = pa.compute_taper_directivity_loss(taper)
    print(f"{name}: efficiency={eff:.1%}, loss={loss:.2f} dB")

pa.plot_comparison_patterns(angles, patterns, title="Taper Comparison")
plt.show()

## 5. Array Geometries

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Rectangular
geom_rect = pa.create_rectangular_array(8, 8, 0.5, 0.5)
pa.plot_array_geometry(geom_rect, title=f"Rectangular ({geom_rect.n_elements} elements)", ax=axes[0,0])

# Triangular
geom_tri = pa.create_triangular_array(8, 8, 0.5)
pa.plot_array_geometry(geom_tri, title=f"Triangular ({geom_tri.n_elements} elements)", ax=axes[0,1])

# Circular
geom_circ = pa.create_circular_array(16, radius=2.0)
pa.plot_array_geometry(geom_circ, title=f"Circular Ring ({geom_circ.n_elements} elements)", ax=axes[0,2])

# Concentric rings
geom_rings = pa.create_concentric_rings_array(3, [8, 12, 16], ring_spacing=0.6)
pa.plot_array_geometry(geom_rings, title=f"Concentric Rings ({geom_rings.n_elements} elements)", ax=axes[1,0])

# Elliptical boundary
geom_ellip = pa.create_elliptical_array(a=3, b=2, dx=0.5, grid_type='rectangular')
pa.plot_array_geometry(geom_ellip, title=f"Elliptical ({geom_ellip.n_elements} elements)", ax=axes[1,1])

# Sparse/thinned
geom_sparse = pa.thin_array_random(geom_rect, thinning_factor=0.5, seed=42)
pa.plot_array_geometry(geom_sparse, title=f"50% Thinned ({geom_sparse.n_elements} elements)", ax=axes[1,2])

plt.tight_layout()
plt.show()

## 6. Null Steering

Place nulls in specific directions to reject interference.

In [ ]:
# Main beam at broadside, nulls at +/-20 degrees
null_directions = [(20, 0), (-20, 0), (15, 90)]  # (theta, phi) in degrees

weights_nulled = pa.null_steering_projection(
    geom, k,
    theta_main_deg=0, phi_main_deg=0,
    null_directions=null_directions
)

# Compute patterns
angles, pattern_uniform, _ = pa.compute_pattern_cuts(
    geom.x, geom.y, np.ones(geom.n_elements), k
)
_, pattern_nulled, _ = pa.compute_pattern_cuts(
    geom.x, geom.y, weights_nulled, k
)

pa.plot_comparison_patterns(
    angles,
    {'Uniform': pattern_uniform, 'With Nulls': pattern_nulled},
    title="Null Steering Comparison"
)

# Mark null positions
for theta_null, phi_null in null_directions:
    plt.axvline(theta_null, color='red', linestyle='--', alpha=0.5)
    plt.axvline(-theta_null, color='red', linestyle='--', alpha=0.5)

plt.show()

# Check null depths
for theta_null, phi_null in null_directions:
    depth = pa.compute_null_depth(weights_nulled, geom, k, theta_null, phi_null, 0, 0)
    print(f"Null at ({theta_null}, {phi_null}) deg: {depth:.1f} dB")

## 7. Multiple Simultaneous Beams

In [ ]:
# Create 3 simultaneous beams
beam_directions = [
    (0, 0),     # Broadside
    (25, 0),    # 25 deg in phi=0
    (25, 180),  # 25 deg in phi=180
]

weights_multi = pa.multi_beam_weights_superposition(
    geom, k, beam_directions,
    amplitudes=[1.0, 0.7, 0.7]  # Slightly lower for side beams
)

# Compute full pattern
theta_full, phi_full, pattern_multi = pa.compute_full_pattern(
    geom.x, geom.y, weights_multi, k,
    n_theta=91, n_phi=181,
    phi_range=(0, np.pi)  # Half-space
)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

angles, E_cut, _ = pa.compute_pattern_cuts(geom.x, geom.y, weights_multi, k)
pa.plot_pattern_2d(angles, E_cut, title="Multi-Beam Pattern (phi=0 cut)", ax=ax1)

pa.plot_pattern_contour(
    np.rad2deg(theta_full), np.rad2deg(phi_full), pattern_multi,
    title="Multi-Beam 2D Pattern", ax=ax2
)

plt.tight_layout()
plt.show()

## 8. Phase Quantization Effects

In [ ]:
# Compare different phase quantization levels
weights_ideal = pa.steering_vector(k, geom.x, geom.y, 30, 0)

patterns_quant = {}
for n_bits in [2, 3, 4, 6, 8]:
    weights_q = pa.quantize_phase(weights_ideal, n_bits)
    angles, E_plane, _ = pa.compute_pattern_cuts(geom.x, geom.y, weights_q, k, 30, 0)
    patterns_quant[f'{n_bits}-bit ({pa.quantization_rms_error(n_bits):.1f} deg RMS)'] = E_plane

# Add ideal
angles, E_ideal, _ = pa.compute_pattern_cuts(geom.x, geom.y, weights_ideal, k, 30, 0)
patterns_quant['Ideal'] = E_ideal

pa.plot_comparison_patterns(angles, patterns_quant, title="Phase Quantization Effects")
plt.show()

## 9. Element Failures and Graceful Degradation

In [ ]:
# Simulate different failure rates
failure_rates = [0, 0.05, 0.10, 0.20]
patterns_fail = {}

weights_nom = pa.steering_vector(k, geom.x, geom.y, 0, 0)
weights_nom *= pa.taylor_taper_2d(Nx, Ny, -30)  # With taper

for rate in failure_rates:
    if rate == 0:
        weights_deg = weights_nom
    else:
        weights_deg, failed = pa.simulate_element_failures(weights_nom, rate, mode='off', seed=42)
    
    angles, E_plane, _ = pa.compute_pattern_cuts(geom.x, geom.y, weights_deg, k)
    patterns_fail[f'{rate*100:.0f}% failures'] = E_plane

pa.plot_comparison_patterns(angles, patterns_fail, title="Element Failure Effects")
plt.show()

## 10. UV-Space Visualization

In [ ]:
# Compute pattern in UV-space
weights_scan = pa.steering_vector(k, geom.x, geom.y, 30, 45)
weights_scan *= pa.taylor_taper_2d(Nx, Ny, -30)

u, v, pattern_uv = pa.compute_pattern_uv_space(
    geom, weights_scan, k,
    n_u=201, n_v=201
)

pa.plot_pattern_uv_space(
    u, v, pattern_uv,
    title="Pattern in UV-Space (scan to 30deg, 45deg)",
    show_visible_region=True,
    show_grating_circles=True,
    dx_wavelengths=0.5,
    dy_wavelengths=0.5
)
plt.show()

## 11. Interactive 3D Visualization (Plotly)

In [ ]:
# 3D spherical pattern plot
theta_3d, phi_3d, pattern_3d = pa.compute_full_pattern(
    geom.x, geom.y, weights_scan, k,
    n_theta=61, n_phi=121
)

fig = pa.plot_pattern_3d_plotly(
    theta_3d, phi_3d, pattern_3d,
    title="3D Radiation Pattern",
    surface_type='spherical'
)
fig.show()

In [ ]:
# Interactive array geometry
geom_cyl = pa.create_cylindrical_array(12, 5, radius=2, height=3)
weights_cyl = pa.steering_vector(k, geom_cyl.x, geom_cyl.y, 30, 0, geom_cyl.z)

fig = pa.plot_array_geometry_3d_plotly(
    geom_cyl, weights_cyl,
    title="Cylindrical Array Geometry",
    show_normals=True
)
fig.show()

## 12. Conformal Array Pattern

In [ ]:
# Create a hemispherical array
geom_sphere = pa.create_spherical_array(
    n_theta=6, n_phi=12,
    radius=5,
    theta_min=0.1, theta_max=np.pi/2
)

# Simple uniform weights
weights_sphere = np.ones(geom_sphere.n_elements, dtype=complex)

# Compute conformal array pattern (accounts for element orientations)
theta_grid, phi_grid = np.meshgrid(
    np.linspace(0, np.pi/2, 91),
    np.linspace(0, 2*np.pi, 181),
    indexing='ij'
)

AF_conformal = pa.array_factor_conformal(
    theta_grid, phi_grid,
    geom_sphere, weights_sphere, k
)

pattern_conf_dB = pa.linear_to_db(np.abs(AF_conformal)**2)
pattern_conf_dB -= np.max(pattern_conf_dB)

fig, ax = plt.subplots(figsize=(10, 8))
pa.plot_pattern_contour(
    np.rad2deg(theta_grid), np.rad2deg(phi_grid), pattern_conf_dB,
    title="Hemispherical Conformal Array Pattern",
    ax=ax
)
plt.show()

## 13. Mutual Coupling Effects

In [ ]:
# Small array for clearer coupling effects
geom_small = pa.create_rectangular_array(8, 8, 0.5, 0.5)
weights_small = pa.steering_vector(k, geom_small.x, geom_small.y, 0, 0)

# Compute coupling matrix
C = pa.mutual_coupling_matrix_theoretical(
    geom_small, k,
    coupling_model='sinc',
    coupling_coeff=0.3
)

# Apply coupling (without and with compensation)
weights_coupled = pa.apply_mutual_coupling(weights_small, C, mode='transmit')
weights_compensated = pa.apply_mutual_coupling(weights_small, C, mode='compensate')

# Compare patterns
angles, p_ideal, _ = pa.compute_pattern_cuts(geom_small.x, geom_small.y, weights_small, k)
_, p_coupled, _ = pa.compute_pattern_cuts(geom_small.x, geom_small.y, weights_coupled, k)
_, p_comp, _ = pa.compute_pattern_cuts(geom_small.x, geom_small.y, weights_compensated, k)

pa.plot_comparison_patterns(
    angles,
    {'Ideal': p_ideal, 'With Coupling': p_coupled, 'Compensated': p_comp},
    title="Mutual Coupling Effects"
)
plt.show()

## 14. Subarray Beamforming

In [ ]:
# Create 16x16 array divided into 4x4 subarrays
arch = pa.create_rectangular_subarrays(
    Nx_total=16, Ny_total=16,
    Nx_sub=4, Ny_sub=4,
    dx=0.5, dy=0.5
)

print(f"Total elements: {arch.geometry.n_elements}")
print(f"Number of subarrays: {arch.n_subarrays}")

# Compute subarray-level steering (quantized phase per subarray)
weights_sub = pa.compute_subarray_weights(
    arch, k,
    theta0_deg=20, phi0_deg=0
)

# Compare to element-level steering
weights_elem = pa.steering_vector(k, arch.geometry.x, arch.geometry.y, 20, 0)

angles, p_elem, _ = pa.compute_pattern_cuts(arch.geometry.x, arch.geometry.y, weights_elem, k, 20, 0)
_, p_sub, _ = pa.compute_pattern_cuts(arch.geometry.x, arch.geometry.y, weights_sub, k, 20, 0)

pa.plot_comparison_patterns(
    angles,
    {'Element-Level': p_elem, 'Subarray-Level': p_sub},
    title="Subarray vs Element-Level Beamforming"
)
plt.show()

## Summary

This notebook demonstrated the key features of the `phased_array` package:

1. **50-100x speedup** with vectorized array factor computation
2. **Multiple array geometries**: rectangular, triangular, circular, conformal, sparse
3. **Amplitude tapering**: Taylor, Chebyshev, Hamming windows for sidelobe control
4. **Null steering**: projection and LCMV methods
5. **Multiple beams**: superposition and orthogonal beamforming
6. **Impairments**: mutual coupling, phase quantization, element failures
7. **Visualization**: 2D plots, UV-space, interactive 3D Plotly

For more details, see the docstrings in each module.